In [74]:
import pandas as pd
import numpy as np
import itertools


In [75]:
### data inladen
wedstrijden = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')
merged_df = pd.merge(wedstrijden, corners, on='wedstrijd_id', how='inner')

In [76]:
merged_df.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'uitslag', 'corners_uitslag',
       'klassement_uitslag', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'score_voor',
       'schepper_pos_rechts', 'positie', 'kop_verplaatsen_achter',
       'kop_veel_verplaatsen', 'kop_hor_verplaatsen', 'goal'],
      dtype='object')

In [77]:
# voor merged_df gaan we alle kolommen met een v in opsplitsen in thuis_kolom en uit_kolom
merged_df[['klassement_thuis', 'klassement_uit']] = merged_df['klassement_uitslag'].str.split('v', expand=True)
merged_df[['uitslag_thuis', 'uitslag_uit']] = merged_df['uitslag'].str.split('v', expand=True)
merged_df[['score_voor_thuis', 'score_voor_uit']] = merged_df['score_voor'].str.split('v', expand=True)

# Convert to numeric
cols_to_convert = [
    'klassement_thuis', 'klassement_uit',
    'uitslag_thuis', 'uitslag_uit',
    'score_voor_thuis', 'score_voor_uit'
]

for col in cols_to_convert:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').astype('Int64')

# drop de kolommen die we niet meer nodig hebben
merged_df = merged_df.drop(columns=['klassement_uitslag', 'uitslag', 'score_voor', "corners_uitslag"])


In [78]:
merged_df.dtypes

wedstrijd_id                       int64
datum                     datetime64[ns]
niveau                            object
schepcorner_id                     int64
ploegnaam                         object
ervaring_schepper                float64
ervaring_kopper                  float64
ervaring_duo                     float64
is_thuisploeg                      int64
kwart                              int64
is_schepper_bank                   int64
is_kopper_bank                     int64
schepper_pos_rechts                int64
positie                            int64
kop_verplaatsen_achter             int64
kop_veel_verplaatsen               int64
kop_hor_verplaatsen                int64
goal                               int64
klassement_thuis                   Int64
klassement_uit                     Int64
uitslag_thuis                      Int64
uitslag_uit                        Int64
score_voor_thuis                   Int64
score_voor_uit                     Int64
dtype: object

In [79]:
wedstrijden.head()

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
0,1,2025-03-15,4,4v1,2v1,1v5
1,2,2024-09-03,4,11v2,2v1,8v3
2,3,2024-09-03,4,4v6,1v1,3v10
3,4,2024-09-03,3,5v3,1v3,4v9
4,5,2024-09-03,4,7v2,0v2,NaN


In [80]:
corners.head()

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
0,1,1,marathon,4.0,4.0,4.0,1,1,1,0,0v0,1,5,0,0,0,1
1,2,1,marathon,4.0,4.0,4.0,1,2,0,0,2v0,1,5,0,0,0,0
2,3,1,poba juniors,3.0,2.0,3.0,0,3,0,0,2v0,0,2,0,0,1,0
3,4,2,mvc rudie,4.0,5.0,4.0,1,3,1,0,4v1,1,5,0,1,1,0
4,5,2,uncle abes patty pounders,1.0,5.0,1.0,0,3,1,0,5v2,1,5,0,0,0,0


### data exploration

In [81]:
### missing values in de merged_df
print(merged_df.isnull().sum()/len(merged_df)*100)

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
dtype: float64


In [82]:
print(wedstrijden.isna().sum()/len(wedstrijden)*100)

wedstrijd_id           0.000000
datum                  0.000000
niveau                 0.000000
uitslag                0.000000
corners_uitslag       33.766234
klassement_uitslag    43.722944
dtype: float64


In [83]:
print(corners.isna().sum()/len(merged_df)*100)

schepcorner_id             0.000000
wedstrijd_id               0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
score_voor                 0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
dtype: float64


### Feature Engineering

In [84]:
#score_verschil_voor
merged_df["score_verschil_voor"] = abs(merged_df["score_voor_thuis"] - merged_df["score_voor_uit"])
#score_eigen
merged_df["score_eigen"] = np.where(merged_df["is_thuisploeg"] == 1, merged_df["score_voor_thuis"], merged_df["score_voor_uit"])
#score_tegenstander
merged_df["score_tegenstander"] = np.where(merged_df["is_thuisploeg"] == 1, merged_df["score_voor_uit"], merged_df["score_voor_thuis"])
# de hoeveelste schepcorner van de wedstrijd
merged_df["wedstrijd_corner_nr"] = merged_df.groupby("wedstrijd_id").cumcount() + 1
# de hoeveelste schepcorner van de ploeg in de wedstrijd
merged_df["ploeg_wedstrijd_corner_nr"] = merged_df.groupby(["wedstrijd_id", "ploegnaam"]).cumcount() + 1

# Functie om periode toe te kennen op basis van maand
def maand_periode(datum):
    maand = datum.month
    if maand in [8, 9]:      # augustus, september
        return 1
    elif maand in [10, 11]:  # oktober, november
        return 2
    elif maand in [12, 1]:   # december, januari
        return 3
    elif maand in [2, 3, 4]: # februari, maart, april
        return 4
    else:
        return None  # mei–juli vallen buiten seizoen

# Toepassen op de basetable
merged_df["periode"] = merged_df["datum"].apply(maand_periode)

# aggregate niveaus, national, kern_hoog, kern_laag
merged_df["niveau"] = merged_df["niveau"].replace({"n1": "nationaal", "n4": "nationaal","p2": "nationaal", 1: "kern_hoog", 2: "kern_hoog", 3: "kern_laag", 4: "kern_laag"})

### creeer voor achter en links midden rechts van positie
merged_df["positie_verticaal_voor"] = np.where(merged_df["positie"].isin([1,2,3]), 1, 0)
merged_df["positie_horizontaal"] = merged_df["positie"].replace({1: "links", 2: "midden", 3: "rechts", 4: "links", 5: "midden", 6: "rechts"})

In [85]:
# lag toevoegen van de vorige schepcorner
lag_df = merged_df[["wedstrijd_id", "schepcorner_id", "datum", "ploegnaam", "goal"]].copy()

lag_df = lag_df.sort_values(["datum", "schepcorner_id"]).reset_index()
lag_df["goal_lag_ploeg"] = lag_df.groupby("ploegnaam")["goal"].shift(1)
lag_df["goal_lag_wedstrijd"] = lag_df.groupby("wedstrijd_id")["goal"].shift(1)

# merge met de basetable
merged_df = merged_df.merge(lag_df[["schepcorner_id", "goal_lag_ploeg", "goal_lag_wedstrijd"]], on="schepcorner_id", how="left")

merged_df[merged_df["ploegnaam"] == "fc spitbulls"][["wedstrijd_id", "schepcorner_id", "datum", "ploegnaam", "goal", "goal_lag_ploeg", "goal_lag_wedstrijd"]].head(20)

,wedstrijd_id,schepcorner_id,datum,ploegnaam,goal,goal_lag_ploeg,goal_lag_wedstrijd
7,3,8,2024-09-03,fc spitbulls,1,0.0,0.0
9,3,10,2024-09-03,fc spitbulls,1,1.0,1.0
75,32,76,2025-04-02,fc spitbulls,1,0.0,NaN
77,32,78,2025-04-02,fc spitbulls,1,1.0,1.0
78,32,79,2025-04-02,fc spitbulls,1,1.0,1.0
79,33,80,2025-03-27,fc spitbulls,1,1.0,NaN
81,33,82,2025-03-27,fc spitbulls,0,1.0,0.0
82,34,83,2025-03-21,fc spitbulls,1,1.0,NaN
85,35,86,2025-03-11,fc spitbulls,1,1.0,1.0
87,36,88,2025-03-06,fc spitbulls,1,0.0,1.0


In [86]:
### klassement imputen
# Rijen met en zonder NA
merged_df['klassement'] = np.where(merged_df['is_thuisploeg'] == 1, merged_df['klassement_thuis'], merged_df['klassement_uit']) 
na_rows = merged_df[merged_df['klassement'].isna()].copy()
notna_rows = merged_df[merged_df['klassement'].notna()].copy()

# Functie om dichtstbijzijnde klassement te zoeken
def impute_klassement(row):
    ploeg = row['ploegnaam']
    datum = row['datum']
    
    kandidaten = notna_rows[notna_rows['ploegnaam'] == ploeg]
    if kandidaten.empty:
        return np.nan  # geen enkele match gevonden
    
    # Bereken absolute tijdsverschil
    kandidaten['datumverschil'] = (kandidaten['datum'] - datum).abs()
    
    # Neem de rij met het kleinste tijdsverschil
    beste_match = kandidaten.loc[kandidaten['datumverschil'].idxmin()]
    
    return beste_match['klassement']

# Imputeren
na_rows['klassement'] = na_rows.apply(impute_klassement, axis=1)

# Combineer terug
df_imputed = pd.concat([notna_rows, na_rows]).sort_index()


In [87]:
print(df_imputed["klassement"].isna().sum()/len(df_imputed)*100)
print(df_imputed["klassement_thuis"].isna().sum()/len(df_imputed)*100)
print(df_imputed["klassement_uit"].isna().sum()/len(df_imputed)*100)


27.69485903814262
41.956882255389715
41.956882255389715


In [88]:
# Start with a copy to avoid SettingWithCopyWarning
df_imputed = df_imputed.copy()

# Impute klassement_thuis
mask_thuis_na = (df_imputed['is_thuisploeg'] == 1) & (df_imputed['klassement_thuis'].isna())
df_imputed.loc[mask_thuis_na, 'klassement_thuis'] = df_imputed.loc[mask_thuis_na, 'klassement']

# Impute klassement_uit
mask_uit_na = (df_imputed['is_thuisploeg'] == 0) & (df_imputed['klassement_uit'].isna())
df_imputed.loc[mask_uit_na, 'klassement_uit'] = df_imputed.loc[mask_uit_na, 'klassement']


In [89]:
# Make a working copy
df_imputed = df_imputed.copy()

# Stap 1: haal bekende klassementen per wedstrijd_id op
klassement_per_match = df_imputed.groupby("wedstrijd_id").agg({
    "klassement_thuis": "first",
    "klassement_uit": "first"
}).reset_index()

# Stap 2: merge deze terug op df_imputed
df_imputed = df_imputed.merge(klassement_per_match, on="wedstrijd_id", suffixes=('', '_from_match'))

# Stap 3: waar klassement_thuis nog ontbreekt, vul in vanuit groepswaarde
mask_thuis_missing = df_imputed["klassement_thuis"].isna() & df_imputed["klassement_thuis_from_match"].notna()
df_imputed.loc[mask_thuis_missing, "klassement_thuis"] = df_imputed.loc[mask_thuis_missing, "klassement_thuis_from_match"]

# Idem voor uit
mask_uit_missing = df_imputed["klassement_uit"].isna() & df_imputed["klassement_uit_from_match"].notna()
df_imputed.loc[mask_uit_missing, "klassement_uit"] = df_imputed.loc[mask_uit_missing, "klassement_uit_from_match"]

# Stap 4: kolommen opruimen
df_imputed.drop(columns=["klassement_thuis_from_match", "klassement_uit_from_match"], inplace=True)


In [90]:
def is_degradatieplaats(rij):
    niveau = str(rij['niveau']).lower()
    klassement = rij['klassement']
    
    # Niveau 4: geen degradatie
    if niveau == '4' or pd.isna(klassement):
        return 0
    # Niveau 1 of 2: degradatie vanaf plaats 13
    elif niveau in ['1', '2']:
        return int(klassement >= 13)
    # Niveau 3: degradatie vanaf plaats 14
    elif niveau == '3':
        return int(klassement >= 14)
    # Niveau n1: degradatie bij plaats 11 of 12
    elif niveau == 'n1':
        return int(klassement in [11, 12])
    # Alle andere niveaus: degradatie vanaf plaats 14
    else:
        return int(klassement >= 14)

In [91]:
def is_promotieplaats(rij):
    klassement = rij['klassement']
    return int(klassement in [1, 2, 3])

In [92]:
# klassement positie 
df_imputed["klassement_veschil"] = abs(df_imputed["klassement_thuis"] - df_imputed["klassement_uit"])
df_imputed["is_degradatieplaats"] = df_imputed.apply(is_degradatieplaats, axis=1)
df_imputed["is_promotieplaats"] = df_imputed.apply(is_promotieplaats, axis=1)


In [93]:
print(df_imputed["is_degradatieplaats"].isna().sum()/len(df_imputed)*100)
print(df_imputed["is_promotieplaats"].isna().sum()/len(df_imputed)*100)
print(df_imputed["klassement_veschil"].isna().sum()/len(df_imputed)*100)
print(df_imputed["klassement"].isna().sum()/len(df_imputed)*100)

0.0
0.0
37.3134328358209
27.69485903814262


In [94]:
# impute klassement_veschil met gemiddelde verschil in een competitie met 14 ploegen
values = list(range(1, 15))  # 1 to 14 inclusive
pairs = list(itertools.combinations(values, 2))

# Compute absolute differences
differences = [abs(a - b) for a, b in pairs]

# Average distance
avg_distance = np.mean(differences)
print(f"Average distance: {avg_distance:.2f}")
df_imputed["klassement_veschil"] = df_imputed["klassement_veschil"].fillna(avg_distance)

# impute klassement met gemiddelde
avg_klassement = sum(values) / len(values)
print(avg_klassement)
df_imputed["klassement"] = df_imputed["klassement"].fillna(avg_klassement)


Average distance: 5.00
7.5


In [95]:
print(df_imputed.isna().sum()/len(df_imputed)*100)

wedstrijd_id                  0.000000
datum                         0.000000
niveau                        0.000000
schepcorner_id                0.000000
ploegnaam                     0.000000
ervaring_schepper            29.519071
ervaring_kopper              29.850746
ervaring_duo                 29.519071
is_thuisploeg                 0.000000
kwart                         0.000000
is_schepper_bank              0.000000
is_kopper_bank                0.000000
schepper_pos_rechts           0.000000
positie                       0.000000
kop_verplaatsen_achter        0.000000
kop_veel_verplaatsen          0.000000
kop_hor_verplaatsen           0.000000
goal                          0.000000
klassement_thuis             28.689884
klassement_uit               29.850746
uitslag_thuis                 0.000000
uitslag_uit                   0.000000
score_voor_thuis              0.000000
score_voor_uit                0.000000
score_verschil_voor           0.000000
score_eigen              

In [96]:
### veel missing values bij ervaring, hoe imputen?
# 1 imputen met vorige wedstrijden, anders met gemiddelde van niveau


### Train test split

In [97]:
basetable = df_imputed.sort_values(["datum", "schepcorner_id"]).reset_index(drop=True)
n = len(basetable)

# Index van splitpunt (80%)
split_index = int(n * 0.8)

# Splitsing
basetable_train = basetable.iloc[:split_index].copy()
basetable_test = basetable.iloc[split_index:].copy()

In [98]:
### imputen ervaringskolommen

In [99]:
def imputatie_rij(rij, ervaring_col, ploegnaam_means, niveau_means, mean_overall):
    ploeg = rij['ploegnaam']
    niveau = rij['niveau']
    
    if not pd.isna(ploegnaam_means.get(ploeg)):
        return ploegnaam_means[ploeg]
    elif not pd.isna(niveau_means.get(niveau)):
        return niveau_means[niveau]
    else:
        return mean_overall


In [100]:
def impute_ervaring_from_train(train_df, target_df, ervaring_col):
    """
    Impute missing values in target_df using statistics calculated from train_df only.
    """
    # Bereken gemiddelden uit trainingsdata
    ploegnaam_means = train_df.groupby('ploegnaam')[ervaring_col].mean()
    niveau_means = train_df.groupby('niveau')[ervaring_col].mean()
    mean_overall = train_df[ervaring_col].mean()

    # Pas imputatie toe op target_df
    df = target_df.copy()
    df[ervaring_col] = df.apply(
        imputatie_rij,
        axis=1,
        ervaring_col=ervaring_col,
        ploegnaam_means=ploegnaam_means,
        niveau_means=niveau_means,
        mean_overall=mean_overall
    )

    return df


In [101]:
train_df_imputed = basetable_train.copy()
test_df_imputed = basetable_test.copy()

for col in ["ervaring_duo", "ervaring_kopper", "ervaring_schepper"]:
    train_df_imputed[col] = impute_ervaring_from_train(
        basetable_train, train_df_imputed, ervaring_col=col
    )[col]

    test_df_imputed[col] = impute_ervaring_from_train(
        basetable_train, test_df_imputed, ervaring_col=col
    )[col]


In [102]:
print(train_df_imputed.isna().sum()/len(train_df_imputed))

wedstrijd_id                 0.000000
datum                        0.000000
niveau                       0.000000
schepcorner_id               0.000000
ploegnaam                    0.000000
ervaring_schepper            0.000000
ervaring_kopper              0.000000
ervaring_duo                 0.000000
is_thuisploeg                0.000000
kwart                        0.000000
is_schepper_bank             0.000000
is_kopper_bank               0.000000
schepper_pos_rechts          0.000000
positie                      0.000000
kop_verplaatsen_achter       0.000000
kop_veel_verplaatsen         0.000000
kop_hor_verplaatsen          0.000000
goal                         0.000000
klassement_thuis             0.358921
klassement_uit               0.365145
uitslag_thuis                0.000000
uitslag_uit                  0.000000
score_voor_thuis             0.000000
score_voor_uit               0.000000
score_verschil_voor          0.000000
score_eigen                  0.000000
score_tegens

In [103]:
# impute lags met gemiddeld scoringspercentage van train set
avg_scoring_perc = train_df_imputed["goal"].mean()
print(avg_scoring_perc)

train_df_imputed["goal_lag_ploeg"] = train_df_imputed["goal_lag_ploeg"].fillna(avg_scoring_perc)
train_df_imputed["goal_lag_wedstrijd"] = train_df_imputed["goal_lag_wedstrijd"].fillna(avg_scoring_perc)
test_df_imputed["goal_lag_ploeg"] = test_df_imputed["goal_lag_ploeg"].fillna(avg_scoring_perc)
test_df_imputed["goal_lag_wedstrijd"] = test_df_imputed["goal_lag_wedstrijd"].fillna(avg_scoring_perc)


0.6639004149377593


In [104]:
train_df_imputed.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'schepper_pos_rechts',
       'positie', 'kop_verplaatsen_achter', 'kop_veel_verplaatsen',
       'kop_hor_verplaatsen', 'goal', 'klassement_thuis', 'klassement_uit',
       'uitslag_thuis', 'uitslag_uit', 'score_voor_thuis', 'score_voor_uit',
       'score_verschil_voor', 'score_eigen', 'score_tegenstander',
       'wedstrijd_corner_nr', 'ploeg_wedstrijd_corner_nr', 'periode',
       'positie_verticaal_voor', 'positie_horizontaal', 'goal_lag_ploeg',
       'goal_lag_wedstrijd', 'klassement', 'klassement_veschil',
       'is_degradatieplaats', 'is_promotieplaats'],
      dtype='object')

In [105]:
### remove unnecessary columns
cols_to_remove = ["wedstrijd_id", "datum", "schepcorner_id", "ploegnaam", "klassement_thuis","klassement_uit", "uitslag_uit", "uitslag_thuis", "positie", "score_voor_thuis", "score_voor_uit"]
train_df_imputed = train_df_imputed.drop(columns=cols_to_remove, axis=1)
test_df_imputed = test_df_imputed.drop(columns=cols_to_remove, axis=1)

In [106]:
### write away
train_df_imputed.to_csv('../../Data/Gold/basetable_train.csv', index=False)
test_df_imputed.to_csv('../../Data/Gold/basetable_test.csv', index=False)
